In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import torch.nn.functional as F
from pytorch_metric_learning import losses
from torchvision.io import read_image
from torchvision import transforms
from tqdm.notebook import tqdm
from PIL import Image
import optuna
from optuna.trial import TrialState

In [2]:
random.seed(2020)

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [4]:
resize_image = transforms.Resize(size=(227, 227))

In [5]:
def calculate_mean(dataset):
    mean = torch.Tensor([0.,0.,0.])
    
    num_samples = len(dataset)
    for i in range(num_samples):
        image, _ = dataset[i]
        image = image.type(torch.FloatTensor) / 255
        
        for j in range(3):
            mean[j] += torch.mean(image[j, :, :])
            
    mean = mean / num_samples
    return mean

In [6]:
def subtract_mean(image, mean):
    normed_image = torch.zeros(image.size())
    image = image.type(torch.FloatTensor) / 255
    
    for j in range(3):
        normed_image[j] = image[j, :, :] - mean[j]
    return normed_image

In [7]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
#         image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]
    
        if self.transform:
            image = self.transform(image, self.mean)

        return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [8]:
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", transform=subtract_mean)
test_data = AdienceDataset("../test.csv", "../cropped_Adience/", transform=subtract_mean)
val_data = AdienceDataset("../val.csv", "../cropped_Adience/", transform=subtract_mean)

In [9]:
# mean = calculate_mean(train_data)
mean = torch.Tensor([0.5062, 0.3581, 0.3099])
mean

tensor([0.5062, 0.3581, 0.3099])

In [10]:
train_data.set_mean(mean)
test_data.set_mean(mean)
val_data.set_mean(mean)

In [11]:
class Net(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
#         sl1 = self.conv11(x)
#         sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        # make alpha trainable
#         x = self.relu((1-alpha)*x + (alpha*sl1))
#         x = self.relu(x + sl1)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
#         sl2 = self.conv22(x)
#         sl2 = self.relu(sl2)
        
        x = self.conv3(x)
#         x = self.relu((1-alpha)*x + (alpha*sl2))
#         x = self.relu(x + sl2)
        x = self.relu(x)
        x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_arcface = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_arcface, x_classification

In [13]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [17]:
model = Net(3, 2).apply(init_weights).to(device)
arc_margin = losses.ArcFaceLoss(2, 512, margin=0.5, scale=30.0).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = opt.SGD(model.parameters(), lr=0.0004197845474223715, momentum=0.9430017883512166, weight_decay=0.013213204541489589)
# optimizer = opt.Adam([{'params': model.parameters()}, {'params': arc_margin.parameters()}], lr=1e-2, weight_decay=0.95)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

optimizer = opt.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

batch_size = 30
num_epochs = 100

In [18]:
best_accu = 0.0

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
    
    running_loss = []
    running_accu = []
    
    model.train()
    for step, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        features, preds = model(images)
        class_loss = criterion(preds, labels)
        
        preds = torch.argmax(preds, 1)
        accuracy = torch.eq(preds, labels).sum() / len(images)
        
        arc_loss = arc_margin(features, labels)
        loss = arc_loss + class_loss
        loss.backward()
        optimizer.step()
        
        running_accu.append(accuracy.cpu().detach().numpy())
        running_loss.append(loss.cpu().detach().numpy())
    lr_scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
    
#     val_loss = []
    val_accu = []

    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)

            features, preds = model(images)

            preds = torch.argmax(preds, 1)
            accuracy = torch.eq(preds, labels).sum() / len(images)

#             arc_out = arc_margin(features, labels)
#             loss = criterion(arc_out, labels)

            val_accu.append(accuracy.cpu().detach().numpy())
#             val_loss.append(loss.cpu().detach().numpy())
    val_accu = np.mean(val_accu)
#     val_loss = np.mean(val_loss)
    print("Val Accuracy: {:.4f}".format(val_accu))
    
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../arc_model.pt")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.4970 - Accuracy: 0.6073


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.7531
Saving best model...


...

Val Accuracy: 0.9232


Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 100/100 - Loss: 0.0007 - Accuracy: 1.0000


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.9232


In [20]:
class SkipNet(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(SkipNet, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv11 = nn.Conv2d(96, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv22 = nn.Conv2d(256, 384, kernel_size=1)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl1 = self.conv11(x)
        sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        # make alpha trainable
#         x = self.relu((1-alpha)*x + (alpha*sl1))
        x = self.relu(x + sl1)
        x = self.pool(x)
        x = self.norm(x)
        
        sl2 = self.conv22(x)
        sl2 = self.relu(sl2)
        
        x = self.conv3(x)
#         x = self.relu((1-alpha)*x + (alpha*sl2))
        x = self.relu(x + sl2)
        x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_arcface = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_arcface, x_classification

In [23]:
model = SkipNet(3, 2).apply(init_weights).to(device)
arc_margin = losses.ArcFaceLoss(2, 512, margin=0.5, scale=30.0).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = opt.SGD(model.parameters(), lr=0.0004197845474223715, momentum=0.9430017883512166, weight_decay=0.013213204541489589)
# optimizer = opt.Adam([{'params': model.parameters()}, {'params': arc_margin.parameters()}], lr=1e-2, weight_decay=0.95)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

optimizer = opt.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

batch_size = 30
num_epochs = 100

In [24]:
best_accu = 0.0

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
    
    running_loss = []
    running_accu = []
    
    model.train()
    for step, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        features, preds = model(images)
        class_loss = criterion(preds, labels)
        
        preds = torch.argmax(preds, 1)
        accuracy = torch.eq(preds, labels).sum() / len(images)
        
        arc_loss = arc_margin(features, labels)
        loss = arc_loss + class_loss
        loss.backward()
        optimizer.step()
        
        running_accu.append(accuracy.cpu().detach().numpy())
        running_loss.append(loss.cpu().detach().numpy())
    lr_scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
    
#     val_loss = []
    val_accu = []

    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)

            features, preds = model(images)

            preds = torch.argmax(preds, 1)
            accuracy = torch.eq(preds, labels).sum() / len(images)

#             arc_out = arc_margin(features, labels)
#             loss = criterion(arc_out, labels)

            val_accu.append(accuracy.cpu().detach().numpy())
#             val_loss.append(loss.cpu().detach().numpy())
    val_accu = np.mean(val_accu)
#     val_loss = np.mean(val_loss)
    print("Val Accuracy: {:.4f}".format(val_accu))
    
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../arc_model_1.pt")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.4964 - Accuracy: 0.6165


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.7339
Saving best model...


...

Val Accuracy: 0.9175


Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 100/100 - Loss: 0.0031 - Accuracy: 0.9999


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.9175


In [22]:
class SkipAlphaNet(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(SkipAlphaNet, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv11 = nn.Conv2d(96, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv22 = nn.Conv2d(256, 384, kernel_size=1)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x, alpha: float):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl1 = self.conv11(x)
        sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        # make alpha trainable
        x = self.relu((1-alpha)*x + (alpha*sl1))
        # x = self.relu(x + sl1)
        x = self.pool(x)
        x = self.norm(x)
        
        sl2 = self.conv22(x)
        sl2 = self.relu(sl2)
        
        x = self.conv3(x)
        x = self.relu((1-alpha)*x + (alpha*sl2))
        # x = self.relu(x + sl2)
        x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_arcface = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_arcface, x_classification

In [27]:
best_accu = 0.0
def objective(trial):
    model = SkipAlphaNet(3, 2)
    model.apply(init_weights)
    model = torch.jit.script(model).to(device)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    mom = trial.suggest_float('momentum', 0.7, 0.99)
    wd = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = opt.SGD(model.parameters(), lr=lr, momentum=mom, weight_decay=wd)
    lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    
    criterion = nn.CrossEntropyLoss()
    arc_margin = losses.ArcFaceLoss(2, 512, margin=0.5, scale=30.0).to(device)
    
    batch_size = trial.suggest_int('batch_size', 10, 100)
    num_epochs = 50
    alpha = trial.suggest_float('alpha', 0.1, 0.9, step=0.1)
    print("Learning rate: "+ str(lr))
    print("Momentum: "+ str(mom))
    print("Weight decay: "+ str(wd))
    print("Alpha: "+ str(alpha))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)

        # training loop
        running_loss = []
        running_accu = []

        model.train()
        for step, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            features, preds = model(images, alpha)
            class_loss = criterion(preds, labels)

            preds = torch.argmax(preds, 1)
            accuracy = torch.eq(preds, labels).sum() / len(images)

            arc_loss = arc_margin(features, labels)
            loss = arc_loss + class_loss
            loss.backward()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        lr_scheduler.step()
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))

        # validation loop
        val_loss = []
        val_accu = []

        model.eval()
        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images, labels = images.to(device), labels.to(device)

                features, preds = model(images, alpha)
                class_loss = criterion(preds, labels)

                preds = torch.argmax(preds, 1)
                accuracy = torch.eq(preds, labels).sum() / len(images)

                arc_loss = arc_margin(features, labels)
                loss = arc_loss + class_loss

                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_accu, epoch)
        if (epoch >= 20 and val_accu < 0.7) or trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    global best_accu
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../arc_model_alpha.pt")
        
    return val_accu

In [28]:
study = optuna.create_study(direction='maximize',
                            study_name='arc-skip-alpha-libfaceid-study',
                            storage='sqlite:///study-arcface.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=20)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-10-31 23:23:08,225] A new study created in RDB with name: arc-skip-alpha-libfaceid-study


Learning rate: 0.004918164975862491
Momentum: 0.9271913750419245
Weight decay: 0.00403573430530614
Alpha: 0.1
Batch size: 28
Number of epochs: 50


Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 1/50 - Loss: 1.5116 - Accuracy: 0.6471


...

Training:   0%|          | 0/822 [00:00<?, ?it/s]

Epoch: 1/50 - Loss: 1.5354 - Accuracy: 0.5552


  0%|          | 0/103 [00:00<?, ?it/s]

[I 2023-11-01 02:29:34,233] Trial 19 pruned. 


Val Loss: 1.5268 - Val Accuracy: 0.6518

Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  12
  Number of complete trials:  8


In [29]:
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
Value:  0.9366438388824463
Params: 
    alpha: 0.5
    batch_size: 12
    learning_rate: 0.0019326822068292092
    momentum: 0.8367867383723058
    weight_decay: 0.0015084020029659609


In [13]:
model = SkipAlphaNet(3, 2).apply(init_weights).to(device)
arc_margin = losses.ArcFaceLoss(2, 512, margin=0.5, scale=30.0).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = opt.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
lr_scheduler = opt.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

batch_size = 30
num_epochs = 50

In [14]:
best_accu = 0.0
alpha = 0.5

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
    
    running_loss = []
    running_accu = []
    
    model.train()
    for step, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        features, preds = model(images, alpha)
        class_loss = criterion(preds, labels)
        
        preds = torch.argmax(preds, 1)
        accuracy = torch.eq(preds, labels).sum() / len(images)
        
        arc_loss = arc_margin(features, labels)
        loss = arc_loss + class_loss
        loss.backward()
        optimizer.step()
        
        running_accu.append(accuracy.cpu().detach().numpy())
        running_loss.append(loss.cpu().detach().numpy())
    lr_scheduler.step()
    print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
    
#     val_loss = []
    val_accu = []

    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images, labels = images.to(device), labels.to(device)

            features, preds = model(images, alpha)

            preds = torch.argmax(preds, 1)
            accuracy = torch.eq(preds, labels).sum() / len(images)

#             arc_out = arc_margin(features, labels)
#             loss = criterion(arc_out, labels)

            val_accu.append(accuracy.cpu().detach().numpy())
#             val_loss.append(loss.cpu().detach().numpy())
    val_accu = np.mean(val_accu)
#     val_loss = np.mean(val_loss)
    print("Val Accuracy: {:.4f}".format(val_accu))
    
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../arc_model_05.pt")

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 1/50 - Loss: 1.5474 - Accuracy: 0.5658


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.6977
Saving best model...


...

Val Accuracy: 0.9266


Training:   0%|          | 0/466 [00:00<?, ?it/s]

Epoch: 50/50 - Loss: 0.0012 - Accuracy: 1.0000


  0%|          | 0/59 [00:00<?, ?it/s]

Val Accuracy: 0.9282


In [ ]:
# skip alpha arcface

Best trial:
Value:  0.9366438388824463
Params: 
    alpha: 0.5
    batch_size: 12
    learning_rate: 0.0019326822068292092
    momentum: 0.8367867383723058
    weight_decay: 0.0015084020029659609